<a href="https://colab.research.google.com/github/DuckShyamalan/COMP6248-Reproducibility-Challenge/blob/main/Defensive_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision
try:
  import torchbearer
except:
  !pip install torchbearer
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import numpy as np
import pickle
from torchbearer.callbacks import LiveLossPlot
from itertools import product
from scipy.ndimage import gaussian_filter
import numpy as np
from matplotlib import pyplot as plt
import random
import torch.nn.functional as F
from torch import nn

In [19]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

In [5]:
#403 fix
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [7]:
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

#split the data
trainset.data = trainset.data[0:27105]
trainset.targets = trainset.targets[0:27105]

In [8]:
# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

In [63]:
class DistillationNetwork(nn.Module):
  def __init__(self):
    super(DistillationNetwork, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, (5, 5), padding=0)
    self.conv2 = nn.Conv2d(32, 64, (3, 3), padding=0)
    self.fc1 = nn.Linear(64 * 5**2, 1200)
    self.fc2 = nn.Linear(1200, 10)

  def forward(self, x):
    x = torch.reshape(x, [-1, 1,28,28])
    out = self.conv1(x)
    out = F.relu(out)
    out = F.max_pool2d(out, (2,2))
    out = self.conv2(out)
    out = F.relu(out)
    out = F.max_pool2d(out, (2,2))
    out = F.dropout(out, 0.2)
    out = out.view(out.shape[0], -1)
    out = self.fc1(out)
    out = F.relu(out)
    out = self.fc2(out)
    if not self.training:
      out = F.softmax(out/50, dim=1)  #authors used T=50
    return out
    

In [64]:
#training cell
model = DistillationNetwork()
loss_function = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)
running_loss = 0.0

trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)

# Provide the data to the trial
trial.with_generators(trainloader, test_generator=testloader)

# Run 2 epochs of training
trial.run(epochs=2)

# test the performance
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

'''for epoch in range(2):
  running_loss = 0.0
  for data in trainloader:
    inputs, labels = data
    #inputs = torch.reshape(inputs, [-1, 1,28,28])
    inputs = inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimiser.zero_grad()

    # forward + loss + backward + optimise (update weights)
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimiser.step()

    # keep track of the loss this epoch
    running_loss += loss.item()
  print("Epoch %d, loss %4.2f" % (epoch, running_loss))
print('**** Finished Training ****')'''


{'test_loss': 2.2779946327209473, 'test_acc': 0.9751999974250793}


'for epoch in range(2):\n  running_loss = 0.0\n  for data in trainloader:\n    inputs, labels = data\n    #inputs = torch.reshape(inputs, [-1, 1,28,28])\n    inputs = inputs.to(device)\n    labels = labels.to(device)\n\n    # zero the parameter gradients\n    optimiser.zero_grad()\n\n    # forward + loss + backward + optimise (update weights)\n    outputs = model(inputs)\n    loss = loss_function(outputs, labels)\n    loss.backward()\n    optimiser.step()\n\n    # keep track of the loss this epoch\n    running_loss += loss.item()\n  print("Epoch %d, loss %4.2f" % (epoch, running_loss))\nprint(\'**** Finished Training ****\')'

96.42%; 97.34% (train, test) accuracy for (3,3) kernel

97.02%; 97.52% for (5,5)